# 特徴選択（ラッパー法）
---
- ラッパー法の一種であるステップワイズ法を実施
- 特徴選択とモデルの学習を繰り返す

## 1. ライブラリの読み込み

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ワインのデータセットを読み込むためのクラス
from sklearn.datasets import load_wine

# ロジスティック回帰モデル
from sklearn.linear_model import SGDClassifier

# ステップワイズ法を実行するためのクラス
# 再帰的な特徴選択を、交差検証法による評価に基づいて行う
# Recursive feature elimination with cross-validation
from sklearn.feature_selection import RFECV

# 多クラス分類の評価指標
from sklearn.metrics import classification_report

## 2. データの読み込み
- データセット：[UCI ML Wine Data Set](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html)
    - 13個の説明変数をもつワインのデータ
    - 3クラスの分類問題
- [各変数の意味](https://www.kaggle.com/code/hnnhytc/data-analysis-with-python)
    - alcohol： アルコール度数
    - malic_acid： リンゴ酸
    - ash： 灰分（かいぶん）
    - alcalinity_of_ash： 灰分のアルカリ度
    - magnesium： マグネシウム
    - total_phenols： 全フェノール含量
    - flavanoids： フラボノイド（ポリフェノールの一種）
    - nonflavanoid_phenols： 非フラボノイドフェノール
    - proanthocyanins： プロアントシアニン（ポリフェノールの一種）
    - color_intensity： 色の濃さ
    - hue： 色相
    - od280/od315_of_diluted_wines： 希釈ワイン溶液のOD280／OD315（＝280nmと315nmの吸光度の比）
    - proline： プロリン（アミノ酸の一種）

In [2]:
# データセットの読み込み
df_X, df_y = load_wine(return_X_y=True, as_frame=True)

# 説明変数を確認
display(df_X.head())
display(df_X.describe())

# 目的変数の内訳を確認
df_y.value_counts()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


target
1    71
0    59
2    48
Name: count, dtype: int64

- クラスごとにデータの件数に偏りがある

## 3. 特徴選択なしで学習

### 3-1. モデルの構築・学習

In [3]:
# 特徴選択用のもの（df_X, df_y）とは別に作成
# 説明変数
X = df_X.values
# 目的変数
y = df_y.values

# 分類問題であるためSGDClassifierを使用
estimator = SGDClassifier(loss='log_loss', max_iter=10000, fit_intercept=True, 
                          random_state=1234, tol=1e-3, )

# 特徴選択なし
# モデルの学習
estimator.fit(X, y)

SGDClassifier(loss='log_loss', max_iter=10000, random_state=1234)

### 3-2. モデルの評価

In [4]:
# 予測結果の取得
y_pred = estimator.predict(X)

# 訓練性能の評価
scores = classification_report(y, y_pred)
print(scores)

              precision    recall  f1-score   support

           0       1.00      0.75      0.85        59
           1       0.58      0.94      0.72        71
           2       0.42      0.17      0.24        48

    accuracy                           0.67       178
   macro avg       0.67      0.62      0.60       178
weighted avg       0.68      0.67      0.63       178



- クラス2のデータに対する性能が低い

## 4. ステップワイズ法による特徴選択

### 4-1. ステップワイズ法の実行
- [RFECV()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html)を用いてステップワイズ法を実行
- 特徴を一部削除しながら学習を行い、交差検証法によりモデルを評価する
- 設定できる[評価指標の一覧](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

In [5]:
# 交差検証を行いつつ、ステップワイズ法による特徴選択を行う
# cvにはFold（=グループ）の数，scoringには評価指標を指定する
# 今回は多クラス分類問題なのでbalanced_accuracyを評価指標に指定
rfecv = RFECV(estimator, cv=10, scoring='balanced_accuracy')

# fitで特徴選択と学習を実行
rfecv.fit(X, y)

RFECV(cv=10,
      estimator=SGDClassifier(loss='log_loss', max_iter=10000,
                              random_state=1234),
      scoring='balanced_accuracy')

### 4-2. 特徴選択の結果を確認

In [6]:
# 特徴のランキングを表示（1が最も重要な特徴）
print('Feature ranking: \n{}'.format(rfecv.ranking_))

Feature ranking: 
[ 2  1 10  3  4  7  1 11  8  1  9  6  5]


In [7]:
# rfecv.support_でランキング1位以外はFalseとするindexを取得できる
# Trueになっている特徴を使用すれば汎化誤差は最小となる
rfecv.support_

array([False,  True, False, False, False, False,  True, False, False,
        True, False, False, False])

In [8]:
# bool型の配列に ~ をつけるとTrueとFalseを反転させることができる
# ここでTrueになっている特徴が、削除してもよい特徴
remove_idx = ~rfecv.support_
remove_idx

array([ True, False,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True])

### 4-3. 特徴の削除
- 今回は以下の3つの変数が残った
    - malic_acid： リンゴ酸
    - flavanoids： フラボノイド（ポリフェノールの一種）
    - color_intensity： 色の濃さ

In [9]:
# 削除してもよい特徴の名前を取得する
remove_feature = df_X.columns[remove_idx]
remove_feature

Index(['alcohol', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols',
       'nonflavanoid_phenols', 'proanthocyanins', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')

In [10]:
# drop関数で特徴を削除
selected_X = df_X.drop(remove_feature, axis=1)
selected_X

,malic_acid,flavanoids,color_intensity
0,1.71,3.06,5.64
1,1.78,2.76,4.38
2,2.36,3.24,5.68
3,1.95,3.49,7.80
4,2.59,2.69,4.32
...,...,...,...
173,5.65,0.61,7.70
174,3.91,0.75,7.30
175,4.28,0.69,10.20
176,2.59,0.68,9.30


### 4-4. 特徴を削除した上で学習

In [11]:
# モデルの学習
estimator.fit(selected_X, y)

# 予測結果の取得
y_pred = estimator.predict(selected_X)

# 訓練性能の評価
scores = classification_report(y, y_pred)
print(scores)



              precision    recall  f1-score   support

           0       0.78      0.98      0.87        59
           1       0.98      0.73      0.84        71
           2       0.94      1.00      0.97        48

    accuracy                           0.89       178
   macro avg       0.90      0.91      0.89       178
weighted avg       0.90      0.89      0.89       178



- バランスよく分類できている